In [2]:
import pandas as pd

train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
answer = train.y
train = train.drop('y', axis=1)
complete = pd.concat([train, test])

column_types = complete.dtypes
categorical_columns = column_types[column_types == 'object'].index.values
numeric = complete.select_dtypes(include=['int64'])
encoded_categorical = pd.get_dummies(complete[categorical_columns])
factorized_categorical = pd.DataFrame()

for column in categorical_columns:
    factorized_categorical[column + '_factor'] = pd.Series(pd.factorize(complete[column], sort=True)[0], complete[column].index.values)
    
complete = pd.concat([numeric, encoded_categorical, factorized_categorical], axis=1)

train = complete.head(len(train))
test = complete.tail(len(test))

In [4]:
from boruta import BorutaPy
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(max_depth=3, n_jobs=-1)
feature_selector = BorutaPy(rf, n_estimators='auto', verbose=2)
feature_selector.fit(train.values, answer.values)

print(feature_selector.support_)
print (feature_selector.ranking_)
print train.columns[feature_selector.support_]

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	588
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	588
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	588
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	588
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	588
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	588
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	588
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	13
Rejected: 	575


/usr/local/lib/python2.7/dist-packages/boruta/boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	9 / 100
Confirmed: 	4
Tentative: 	9
Rejected: 	575
Iteration: 	10 / 100
Confirmed: 	4
Tentative: 	9
Rejected: 	575
Iteration: 	11 / 100
Confirmed: 	4
Tentative: 	9
Rejected: 	575
Iteration: 	12 / 100
Confirmed: 	12
Tentative: 	1
Rejected: 	575
Iteration: 	13 / 100
Confirmed: 	12
Tentative: 	1
Rejected: 	575
Iteration: 	14 / 100
Confirmed: 	12
Tentative: 	1
Rejected: 	575
Iteration: 	15 / 100
Confirmed: 	12
Tentative: 	1
Rejected: 	575
Iteration: 	16 / 100
Confirmed: 	13
Tentative: 	0
Rejected: 	575


BorutaPy finished running.

Iteration: 	17 / 100
Confirmed: 	13
Tentative: 	0
Rejected: 	575
[False False False False False False False False False False False False
 False False False False False False False  True False False False False
 False False False False False False False False False False False False
 False False False False False False False False  True False False False
 False False False False False False False False False False False False
 False False False False

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm.sklearn import LGBMRegressor
from sklearn.model_selection import RepeatedKFold

sfs = SequentialFeatureSelector(estimator=LGBMRegressor(n_estimators=50, num_leaves=12), 
           k_features=12,
           forward=True,
           floating=True,
           scoring='r2',
           cv=RepeatedKFold(),
           verbose=2)

sfs.fit(train.values, answer.values)

print('Best combination (r2: %.3f): %s\n' % (sfs.k_score_, sfs.k_feature_idx_))
print('All subsets:\n', sfs.subsets_)
print(train.columns[list(sfs.k_feature_idx_)].values)


[2017-06-24 13:14:24] Features: 1/12 -- score: 0.552225316132
[2017-06-24 13:19:37] Features: 2/12 -- score: 0.563245311535
[2017-06-24 13:25:04] Features: 3/12 -- score: 0.570032135681
[2017-06-24 13:47:38] Features: 7/12 -- score: 0.575109413082
[2017-06-24 13:53:15] Features: 8/12 -- score: 0.575232841312
[2017-06-24 13:58:56] Features: 9/12 -- score: 0.574927703039
[2017-06-24 14:04:37] Features: 10/12 -- score: 0.574913618331
[2017-06-24 14:10:20] Features: 11/12 -- score: 0.575330974174

Best combination (r2: 0.575): (19, 580, 37, 135, 8, 459, 50, 115, 212, 356, 410, 539)

('All subsets:\n', {1: {'avg_score': 0.55222531613172154, 'feature_idx': (580,), 'cv_scores': array([ 0.45773636,  0.60434349,  0.56582712,  0.57316925,  0.55915452,
        0.57832289,  0.46715352,  0.58363444,  0.51830819,  0.61029127,
        0.56868755,  0.5862065 ,  0.56235437,  0.60973758,  0.43341399,
        0.49078748,  0.60997053,  0.58800185,  0.56834705,  0.48695179,
        0.57889399,  0.47781512,  0.60063503,  0.55719493,  0.5556526 ,
        0.52853732,  0.57471756,  0.5000217 ,  0.56032968,  0.58137151,
        0.60938433,  0.57298098,  0.60656898,  0.40593335,  0.55976077,
        0.57161778,  0.6264098 ,  0.54939397,  0.46325954,  0.56017924,
        0.42712942,  0.61189744,  0.55282427,  0.54263031,  0.63488867,
        0.45736006,  0.59182337,  0.5996472 ,  0.61688947,  0.5131177 ])}, 2: {'avg_score': 0.56324531153482149, 'feature_idx': (115, 580), 'cv_scores': array([ 0.56177517


[2017-06-24 14:16:03] Features: 12/12 -- score: 0.575067246043

In [7]:
train.columns[[19, 580, 37, 135, 8, 459, 50, 115, 212, 356, 410]].values

array(['X29', 'X0_factor', 'X47', 'X147', 'X17', 'X2_aj', 'X60', 'X127',
       'X227', 'X372', 'X0_o'], dtype=object)

In [139]:
train.columns[[192, 265, 331, 76, 335, 273, 274, 275, 341, 282, 28, 218, 32, 112, 37, 294, 233, 175, 368, 369, 115, 309, 118, 255]]

Index([u'X207', u'X280', u'X347', u'X87', u'X351', u'X288', u'X289', u'X290',
       u'X357', u'X297', u'X38', u'X233', u'X42', u'X124', u'X47', u'X310',
       u'X248', u'X189', u'X385', u'X0_factor', u'X127', u'X325', u'X130',
       u'X270'],
      dtype='object')

In [ ]:
[2017-06-24 13:14:24] Features: 1/12 -- score: 0.552225316132
[2017-06-24 13:19:37] Features: 2/12 -- score: 0.563245311535
[2017-06-24 13:25:04] Features: 3/12 -- score: 0.570032135681
[2017-06-24 13:47:38] Features: 7/12 -- score: 0.575109413082
[2017-06-24 13:53:15] Features: 8/12 -- score: 0.575232841312
[2017-06-24 13:58:56] Features: 9/12 -- score: 0.574927703039
[2017-06-24 14:04:37] Features: 10/12 -- score: 0.574913618331
[2017-06-24 14:10:20] Features: 11/12 -- score: 0.575330974174
Best combination (r2: 0.575): (19, 580, 37, 135, 8, 459, 50, 115, 212, 356, 410, 539)

('All subsets:\n', {1: {'avg_score': 0.55222531613172154, 'feature_idx': (580,), 'cv_scores': array([ 0.45773636,  0.60434349,  0.56582712,  0.57316925,  0.55915452,
        0.57832289,  0.46715352,  0.58363444,  0.51830819,  0.61029127,
        0.56868755,  0.5862065 ,  0.56235437,  0.60973758,  0.43341399,
        0.49078748,  0.60997053,  0.58800185,  0.56834705,  0.48695179,
        0.57889399,  0.47781512,  0.60063503,  0.55719493,  0.5556526 ,
        0.52853732,  0.57471756,  0.5000217 ,  0.56032968,  0.58137151,
        0.60938433,  0.57298098,  0.60656898,  0.40593335,  0.55976077,
        0.57161778,  0.6264098 ,  0.54939397,  0.46325954,  0.56017924,
        0.42712942,  0.61189744,  0.55282427,  0.54263031,  0.63488867,
        0.45736006,  0.59182337,  0.5996472 ,  0.61688947,  0.5131177 ])}, 2: {'avg_score': 0.56324531153482149, 'feature_idx': (115, 580), 'cv_scores': array([ 0.56177517,  0.58334331,  0.59649794,  0.49923934,  0.57403707,
        0.5229952 ,  0.59483355,  0.49532178,  0.58064456,  0.60851204,
        0.60247187,  0.58322118,  0.59645149,  0.4633063 ,  0.57497371,
        0.61713266,  0.56666311,  0.49912273,  0.59745674,  0.54589027,
        0.58460466,  0.59451178,  0.58561134,  0.6133732 ,  0.45718818,
        0.64251357,  0.52909752,  0.49739366,  0.57566877,  0.56795067,
        0.53488   ,  0.48738491,  0.62832262,  0.58537023,  0.56257611,
        0.59706369,  0.62930933,  0.56837883,  0.47884614,  0.5503758 ,
        0.60060809,  0.5980418 ,  0.55394421,  0.60217537,  0.4677501 ,
        0.58761067,  0.61235413,  0.59480044,  0.5244288 ,  0.48624092])}, 3: {'avg_score': 0.57003213568122635, 'feature_idx': (115, 580, 37), 'cv_scores': array([ 0.61123875,  0.5629615 ,  0.57586973,  0.61446256,  0.46897846,
        0.46594882,  0.54181029,  0.59815053,  0.67361023,  0.57170543,
        0.58121567,  0.59172907,  0.56759935,  0.4660117 ,  0.65913229,
        0.48921682,  0.59575325,  0.57323651,  0.61926609,  0.58195033,
        0.60283595,  0.62604608,  0.46228373,  0.60383672,  0.55850046,
        0.5969054 ,  0.57528628,  0.60679376,  0.46811262,  0.60401273,
        0.59702935,  0.49251719,  0.63982211,  0.55810647,  0.56293728,
        0.61105841,  0.60538251,  0.44926438,  0.60270554,  0.59097978,
        0.60548294,  0.46906786,  0.56170722,  0.60822671,  0.60607861,
        0.49050579,  0.59464598,  0.55411145,  0.60434229,  0.58317382])}, 4: {'avg_score': 0.57366520667562038, 'feature_idx': (115, 580, 37, 459), 'cv_scores': array([ 0.5844313 ,  0.56694539,  0.4758876 ,  0.6339951 ,  0.59356431,
        0.6056452 ,  0.55999425,  0.65373316,  0.61368666,  0.44812327,
        0.60507731,  0.60310298,  0.59706572,  0.43565817,  0.65197856,
        0.47789888,  0.64223897,  0.52607116,  0.61580062,  0.61517583,
        0.58647201,  0.58436731,  0.57978193,  0.44957188,  0.66739005,
        0.61286325,  0.59563096,  0.59754897,  0.47020533,  0.59983446,
        0.61395565,  0.60515018,  0.58250376,  0.48524978,  0.56773573,
        0.48956041,  0.59307764,  0.56016273,  0.59182682,  0.61262384,
        0.57754493,  0.60521802,  0.6279186 ,  0.46400912,  0.60241946,
        0.61058058,  0.61829893,  0.53349974,  0.59921959,  0.49296422])}, 5: {'avg_score': 0.57354146011826823, 'feature_idx': (19, 115, 580, 37, 459), 'cv_scores': array([ 0.61495901,  0.57211807,  0.47774313,  0.59640218,  0.60979777,
        0.48963106,  0.61429969,  0.5707204 ,  0.60240642,  0.5904736 ,
        0.58433608,  0.62242923,  0.46202912,  0.61555524,  0.58218453,
        0.60837112,  0.56854091,  0.61400692,  0.6209447 ,  0.46044061,
        0.55102352,  0.62157877,  0.46047502,  0.57790373,  0.66508628,
        0.46277491,  0.56591897,  0.60929961,  0.58522181,  0.64731093,
        0.57890221,  0.59791083,  0.66029068,  0.59882811,  0.43546974,
        0.63301035,  0.57868843,  0.60134887,  0.58161082,  0.47343546,
        0.56681308,  0.57961191,  0.50863834,  0.57534775,  0.62137681,
        0.50070452,  0.57899252,  0.61384416,  0.55402075,  0.61424438])}, 6: {'avg_score': 0.57468549000154034, 'feature_idx': (115, 19, 580, 37, 135, 459), 'cv_scores': array([ 0.55937379,  0.4499859 ,  0.61605641,  0.61108024,  0.63390933,
        0.47916971,  0.48124389,  0.63497583,  0.62256918,  0.64732897,
        0.59982871,  0.57402766,  0.6138047 ,  0.64410926,  0.44376043,
        0.46467951,  0.66535536,  0.63514334,  0.59259778,  0.53181902,
        0.61258318,  0.50475668,  0.5189922 ,  0.62307846,  0.59516085,
        0.63194835,  0.49655002,  0.58145835,  0.55075102,  0.60911388,
        0.64496388,  0.56599436,  0.58708286,  0.4553248 ,  0.63704453,
        0.5835094 ,  0.63353814,  0.61664448,  0.58124381,  0.47178691,
        0.60809208,  0.52039962,  0.56805292,  0.6255614 ,  0.53701553,
        0.61248484,  0.63457039,  0.5702286 ,  0.59243782,  0.46308612])}, 7: {'avg_score': 0.57510941308247077, 'feature_idx': (19, 115, 580, 37, 135, 212, 459), 'cv_scores': array([ 0.6281794 ,  0.44968969,  0.64273048,  0.58622845,  0.55693489,
        0.57916663,  0.62417107,  0.55916495,  0.49079234,  0.63716928,
        0.58665394,  0.65541411,  0.47449804,  0.59813784,  0.55049366,
        0.59389583,  0.4812452 ,  0.59626342,  0.56347117,  0.63529378,
        0.58144343,  0.61722114,  0.49373814,  0.5657777 ,  0.60471629,
        0.62792492,  0.43421074,  0.61336697,  0.6485999 ,  0.57728627,
        0.58346514,  0.58542198,  0.61050809,  0.65256134,  0.45645023,
        0.62533934,  0.56376141,  0.641769  ,  0.44131958,  0.59658226,
        0.61025683,  0.57885391,  0.59921212,  0.46636956,  0.61383278,
        0.60116715,  0.43063655,  0.62994316,  0.62804719,  0.58609335])}, 8: {'avg_score': 0.57523284131165653, 'feature_idx': (115, 19, 580, 37, 135, 212, 410, 459), 'cv_scores': array([ 0.4402332 ,  0.60102772,  0.61255663,  0.61406572,  0.61093858,
        0.57082316,  0.48879423,  0.58214742,  0.60704428,  0.62566707,
        0.60195903,  0.6292896 ,  0.55795114,  0.48014047,  0.61115453,
        0.54521354,  0.57764063,  0.63312141,  0.48253857,  0.6326461 ,
        0.5796202 ,  0.62040397,  0.60732994,  0.43693211,  0.640935  ,
        0.60450365,  0.61227863,  0.62449204,  0.43596079,  0.61822527,
        0.59577515,  0.58442072,  0.63098162,  0.44801372,  0.59683857,
        0.45278703,  0.62504525,  0.61237243,  0.61389228,  0.55370522,
        0.60106668,  0.5930254 ,  0.47555282,  0.59377271,  0.61700451,
        0.57727364,  0.54324555,  0.62720983,  0.65363054,  0.48039377])}, 9: {'avg_score': 0.57492770303866036, 'feature_idx': (19, 580, 37, 135, 459, 50, 115, 212, 410), 'cv_scores': array([ 0.61567251,  0.45632314,  0.61758047,  0.59420324,  0.58759072,
        0.61201118,  0.46911845,  0.59506148,  0.62133495,  0.58866715,
        0.63908236,  0.60605131,  0.47455383,  0.60470779,  0.55488871,
        0.63670021,  0.47900678,  0.52584472,  0.58001024,  0.643549  ,
        0.44472115,  0.58424371,  0.61720895,  0.6225953 ,  0.60819528,
        0.56575361,  0.50114582,  0.54945513,  0.63958385,  0.6004183 ,
        0.64246754,  0.43802478,  0.591867  ,  0.60674355,  0.61207356,
        0.56663553,  0.55176756,  0.62273919,  0.50985247,  0.61088226,
        0.63944389,  0.60453669,  0.59872801,  0.46061555,  0.57961425,
        0.59257119,  0.64708672,  0.59796558,  0.45132116,  0.58616936])}, 10: {'avg_score': 0.5749136183307535, 'feature_idx': (19, 580, 37, 135, 459, 50, 115, 212, 356, 410), 'cv_scores': array([ 0.48061066,  0.5673321 ,  0.61189997,  0.59601251,  0.61908861,
        0.49811207,  0.58963288,  0.60300701,  0.5616471 ,  0.61480787,
        0.59575163,  0.45656099,  0.61576539,  0.56404495,  0.62074489,
        0.5900383 ,  0.56525376,  0.4900749 ,  0.56240752,  0.66040366,
        0.63569207,  0.608344  ,  0.57210579,  0.48003278,  0.577626  ,
        0.63895813,  0.58320238,  0.65356924,  0.61329275,  0.41342526,
        0.44869333,  0.62873989,  0.58592267,  0.61730687,  0.60342688,
        0.62538827,  0.55385973,  0.61253972,  0.56217524,  0.49853925,
        0.43837925,  0.58014278,  0.63512624,  0.62778184,  0.61543899,
        0.60864734,  0.59598245,  0.58407115,  0.49129344,  0.59278036])}, 11: {'avg_score': 0.57533097417399415, 'feature_idx': (19, 580, 37, 135, 8, 459, 50, 115, 212, 356, 410), 'cv_scores': array([ 0.44998658,  0.62451651,  0.57160808,  0.59288953,  0.6236891 ,
        0.63986665,  0.59343673,  0.56161006,  0.48866685,  0.59174386,
        0.5938804 ,  0.6111256 ,  0.46603732,  0.63986206,  0.57524738,
        0.61627063,  0.58240704,  0.44321683,  0.64003434,  0.58014334,
        0.46015898,  0.61289307,  0.59492998,  0.61298772,  0.59343247,
        0.53568439,  0.62139973,  0.67027617,  0.4415832 ,  0.63226016,
        0.57168056,  0.44357562,  0.61970686,  0.6215173 ,  0.62076492,
        0.60415427,  0.57348652,  0.4558527 ,  0.57153511,  0.6614039 ,
        0.61548252,  0.44783861,  0.587307  ,  0.6399485 ,  0.59345568,
        0.66085366,  0.58067714,  0.58595907,  0.42526832,  0.62423569])}, 12: {'avg_score': 0.57506724604323134, 'feature_idx': (19, 580, 37, 135, 8, 459, 50, 115, 212, 356, 410, 539), 'cv_scores': array([ 0.48088484,  0.64026823,  0.60583286,  0.55146939,  0.59894861,
        0.55209295,  0.58581156,  0.61138857,  0.49078266,  0.63864102,
        0.59777457,  0.48703674,  0.60704759,  0.54054716,  0.63133146,
        0.59198948,  0.60702273,  0.47935482,  0.58801367,  0.60784573,
        0.59137793,  0.50120363,  0.6682734 ,  0.57582346,  0.52944409,
        0.59851084,  0.62587196,  0.60285987,  0.58619573,  0.47479866,
        0.62296166,  0.60800599,  0.63171432,  0.59041283,  0.4462975 ,
        0.54874756,  0.48609938,  0.60026912,  0.5933344 ,  0.63226252,
        0.65043989,  0.61698118,  0.45995377,  0.55630106,  0.59771019,
        0.62676403,  0.49095585,  0.59148092,  0.53650124,  0.61772468])}})
Index([u'X29', u'X0_factor', u'X47', u'X147', u'X17', u'X2_aj', u'X60',
       u'X127', u'X227', u'X372', u'X0_o', u'X5_w'],
      dtype='object')